In [1]:
import nltk
import csv
import math
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import OrderedDict
import operator

In [2]:
#Import all Documents
D={}
for x in range(50):
    x=x+1
    file="Documents/"+str(x)+".txt"
    D[x]=open(file).read()

In [3]:
#appended all documents to allDoc
allDoc=""
for x in range(50):
    x=x+1
    allDoc=allDoc+" \n"+D[x]

#----------------------------------------------Tokenization--------------------------------------------------------------------------------------------#

In [4]:
#Tokens
tokens=nltk.word_tokenize(allDoc)
print(len(tokens))

144647


In [5]:
# Unique Tokens 
tokens=list(set(tokens))
print(len(tokens))

12199


In [6]:
#Remove Special characters
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
tokens=[x.translate(removetable) for x in tokens]

In [7]:
#Importing Stoplist
stopWord=open("Stopword-List.txt").read()
stopWord=nltk.word_tokenize(stopWord)

In [8]:
#Decapitalized
tokens=[element.lower() for element in tokens]
print(len(tokens))

12199


In [9]:
# Removing StopWords
tokens=[x for x in tokens if x.isalnum() and x not in stopWord]

In [10]:
tokens=[element.lower() for element in tokens]
print(len(tokens))

12031


In [11]:
#Sorted Tokens
tokens=sorted(tokens)
print(len(tokens))

12031


#--------------------------------------END------------------------------------------------------------------------------------#

#----------------------------------DOCument Wise Tokenization----------------------------------------------------------------#

In [12]:
#Document wise Tokenization
docToken={}
for x in range(50):
    x=x+1
    docToken[x]=nltk.word_tokenize(D[x])

In [13]:
#unique Token Doc wise
#for x in range(50):
 #   x=x+1
  #  docToken[x]=set(docToken[x])
   # docToken[x]=list(docToken[x])

In [14]:
#Remove Special characters Doc Wise
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
for x in range(50):
    x=x+1
    docToken[x]=[y.translate(removetable) for y in docToken[x]]

In [15]:
#doc wise sorted
for x in range(50):
    x=x+1
    docToken[x]=sorted(docToken[x])

In [16]:
#Decaptilized Doc Wise
for x in range(1,50):
    docToken[x]=[element.lower() for element in docToken[x]]

In [17]:
for x in range(50):
    x=x+1
    docToken[x]=[y for y in docToken[x] if y.isalnum() and y not in stopWord]

#--------------------------------------END------------------------------------------------------------------------------------#

#-----------------------------------------------Word Frequency-----------------------------------------------------------#

### TF(term) = # of times the term appears in document / total # of terms in document

In [18]:
#unique Tokens    ??Test
#fdist=nltk.FreqDist(tokens)
#print(len(fdist))
#fdist.most_common(10)

# TF-iDF

In [19]:
docV={}

In [20]:
for x in range(1,50):
    docV[x]=dict.fromkeys(tokens,0) 

In [21]:
#Term Frequency in  a document
for x in range(1,50):
    for word in docToken[x]:
        docV[x][word]+=1    

In [22]:
print(docV[23]['crowd'])

2


### Term Frequency (tf)

In [23]:
#tf
tfDocV={}
for x in range(1,50):
    tfDocV[x]={}
    for word,count in docV[x].items():
        tfDocV[x][word]=count

In [24]:
print(tfDocV[23]['crowd'])

2


### inverse Document Frequency (idf)

In [25]:
#unique Token Doc wise
for x in range(1,50):
    docToken[x]=set(docToken[x])
    docToken[x]=list(set(docToken[x]))

In [26]:
wordDcount=dict.fromkeys(tokens,0)
for word in tokens:
    for x in range(1,50):
        if word in docToken[x]:
            wordDcount[word]+=1

In [27]:
print(wordDcount['was'])

98


In [28]:
idfDict = {}
for word in tokens:
    if wordDcount[word]>0:
        count=wordDcount[word]
        if count>50:
            count=50
    idfDict[word]=math.log(50/count)

## tfidf

In [29]:
tfidf={}
for x in range(1,50):
    tfidf[x]={}
    for word in docV[x]:
        tfidf[x][word]=tfDocV[x][word]*idfDict[word]

In [30]:
print(tfidf[37]['crowd'])

5.497744391244931


## Document Vector

## query Processing

In [31]:
query="crowd busy"

In [32]:
qt=nltk.word_tokenize(query)
print(qt)

['crowd', 'busy']


In [33]:
#Remove Special characters
removetable=str.maketrans("", "", "'!@#$%^&*()_=-\|][:';:,<.>/?`~")
qt=[x.translate(removetable) for x in qt]

In [34]:
#Decapitalized
qt=[element.lower() for element in qt]
print(len(qt))

2


In [35]:
qt=[y for y in qt if y.isalnum() and y not in stopWord]
qt=list(set(qt))
qt

['busy', 'crowd']

In [36]:
qtV=dict.fromkeys(tokens,0)
for word in qt:
    qtV[word]+=1    

In [37]:
qtV['busy']

1

In [38]:
#quert idf
for words in qtV:
    qtV[words]=qtV[words]*idfDict[word]
    

In [39]:
qtV['busy']

1.8325814637483102

## Cosine Similarity

In [40]:
res={}
temp=0
vec1=np.array([list(qtV.values())])
for x in range(1,50):
    vec2=np.array([list(tfidf[x].values())])
    if cosine_similarity(vec1,vec2)>0:
        temp=cosine_similarity(vec1,vec2)[0][0]
        res[x]=temp

In [41]:
len(res)

12

In [42]:
sorted(res.items(), key=operator.itemgetter(1), reverse=True)

[(23, 0.029240146588360154),
 (37, 0.01697641088572612),
 (7, 0.016882130700743042),
 (17, 0.016602833699056747),
 (12, 0.014605634187455514),
 (46, 0.014500302734983232),
 (3, 0.012995153918962502),
 (39, 0.012112311957513212),
 (6, 0.010355684388672574),
 (27, 0.010140927456497743),
 (44, 0.00946641462360756),
 (20, 0.007915184672586376)]